##　ニューラルネットワーク

損失関数で求めた誤差を各パラメータに逆伝播して損失に対する勾配をもとめ、勾配に応じてパラメータを更新する

線形関数（各パラメータを含む）に活性化関数（非線形関数）を適用することで、複雑な関数として動く
o = tanh(wx + b)

oとｘはスカラーでもベクトルでも
ｗもスカラーでも行列でも
ｂはスカラーかベクトル（入力と出力の次元は同じ）

誤差関数の理解

線形モデルでは誤差関数が凸型だが、ニューラルネットでは各パラメータに唯一の解が無く、凸上にならない

活性化関数の意味

線形変換では10点々での評価でも11点を出すなど出力を特定の範囲に制限できない

1/(1+e** -x)を計算するnn.Sigmoidやtanhなどを用いることで解決できる
xが負の無限大に近づくと0または―1へ、正の無限大に近づくと１になる2

最適な活性化関数の選択

活性化関数なしでは繰り返し使用しても同じ形式の関数が得られる（アフィン変換）拡大回転等のみ
活性化関数は微分可能

活性化関数は入力に対して感度の高い領域を保有し、入力値の重要な変化に応じ変化も大きくなる。訓練に必要な特徴
また入力値の変化に出力が変化しないような飽和領域も保有している

## nnモジュール

In [75]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim

t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [76]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 1, 10,  2,  8,  5,  9,  6,  0,  4]), tensor([3, 7]))

In [77]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [78]:
import torch.nn as nn
linear_model = nn.Linear(1, 1)#コンストラクタに引数を与える
linear_model(t_un_val)

tensor([[5.8782],
        [1.9026]], grad_fn=<AddmmBackward0>)

In [79]:
#インスタンスと引数を渡すと、同じ引数をもつfowardが実行される
#下は同じだが、foward()は呼ばない

#y = model(x)
#y = model.foward(x)

nnで線形モデル作成（無駄だが）

In [80]:
import torch.nn as nn
#Linearの引数は三つ（入力特徴量の次元、出力特徴量の次元、バイアルの有無（デフォルトはTrue））
linear_model = nn.Linear(1, 1)
linear_model(t_un_val)

tensor([[ 1.6335],
        [-0.2067]], grad_fn=<AddmmBackward0>)

In [81]:
linear_model.weight

Parameter containing:
tensor([[0.3062]], requires_grad=True)

In [82]:
linear_model.bias

Parameter containing:
tensor([-0.8742], requires_grad=True)

In [83]:
#一部部省略して入力が可能、これはバッチ次元（０）を省略している
x = torch.ones(1)
linear_model(x)

tensor([-0.5680], grad_fn=<AddBackward0>)

In [84]:
# nn.Moduleとそのサブクラスは複数の入出力を同時に扱える
# 入力の０次元はバッチ数を想定
# バッチ数：１０、特量量の数：１

x = torch.ones(10, 1)
linear_model(x)

tensor([[-0.5680],
        [-0.5680],
        [-0.5680],
        [-0.5680],
        [-0.5680],
        [-0.5680],
        [-0.5680],
        [-0.5680],
        [-0.5680],
        [-0.5680]], grad_fn=<AddmmBackward0>)

バッチの最適化

バッチ処理をする理由はリソースの飽和を防ぐ、バッチ全体の統計情報使用しているモデルもありバッチサイズが大きくなると性能向上する

In [107]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # 余分な次元の追加
t_u = torch.tensor(t_u).unsqueeze(1)

In [108]:
liner_model = nn.Linear(1, 1)
#optimaizerの引数はパラメータとlr
optimaizer = optim.SGD(
    linear_model.parameters(),#linear_modelのパラメータを
    lr = 1e-2)

In [109]:
linear_model.parameters()

<generator object Module.parameters at 0x7f1b40de5f20>

In [110]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[5.1138]], requires_grad=True),
 Parameter containing:
 tensor([-16.2474], requires_grad=True)]

In [111]:
#modelは引数にいれ使用
#損失関数もできものを使用
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs +1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [112]:
#損失関数はnn.MSELossで既存のものを使用
#nn.はインスタンスを作成し関数として使用
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 76.2894, Validation loss 268.3383
Epoch 1000, Training loss 5.6591, Validation loss 19.5508
Epoch 2000, Training loss 3.5314, Validation loss 5.6341
Epoch 3000, Training loss 3.0927, Validation loss 3.3478

Parameter containing:
tensor([[5.1324]], requires_grad=True)
Parameter containing:
tensor([-16.3475], requires_grad=True)


## ニューラルネット構築

In [113]:
#nnモジュールの連結にnn.sequentialを使用
#13は適当入出力は合わせる

seq_model = nn.Sequential(
    nn.Linear(1, 13),
    nn.Tanh(),
    nn.Linear(13, 1))

seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [114]:
#parametersはoptimaizerが取得するパラメータ
#bavkfowrdで計算されるもの
[param.shape for param in seq_model.parameters()]

[torch.Size([13, 1]), torch.Size([13]), torch.Size([1, 13]), torch.Size([1])]

In [115]:
#パラメータに名前があり識別できる
for name, param in seq_model.named_parameters():
    print(name, param.shape)

0.weight torch.Size([13, 1])
0.bias torch.Size([13])
2.weight torch.Size([1, 13])
2.bias torch.Size([1])


In [116]:
#自由に名前を付けることも可能
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([("hidden_Linear", nn.Linear(1, 8)),
                                      ("hiddrn_activation", nn.Tanh()),
                                      ("output_Linear", nn.Linear(8, 1))
                                     ]))

seq_model

Sequential(
  (hidden_Linear): Linear(in_features=1, out_features=8, bias=True)
  (hiddrn_activation): Tanh()
  (output_Linear): Linear(in_features=8, out_features=1, bias=True)
)

In [117]:
seq_model.output_Linear

Linear(in_features=8, out_features=1, bias=True)

In [118]:
seq_model.output_Linear.bias

Parameter containing:
tensor([0.1516], requires_grad=True)

In [121]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(n_epochs = 5000,
              optimizer = optimizer,
              model = seq_model,
              loss_fn = nn.MSELoss(),
              t_u_train = t_un_train,
              t_u_val = t_un_val,
              t_c_train = t_c_train,
              t_c_val = t_c_val)

Epoch 1, Training loss 35.3453, Validation loss 242.9655
Epoch 1000, Training loss 6.4881, Validation loss 60.8218
Epoch 2000, Training loss 3.1714, Validation loss 25.5596
Epoch 3000, Training loss 2.5908, Validation loss 16.9956
Epoch 4000, Training loss 2.9586, Validation loss 15.3346
Epoch 5000, Training loss 2.6761, Validation loss 13.4934


In [122]:
print("output", seq_model(t_un_val))
print("answer", t_c_val)
print("hidden", seq_model.hidden_Linear.weight.grad)

output tensor([[23.8691],
        [-1.5857]], grad_fn=<AddmmBackward0>)
answer tensor([[28.],
        [-4.]])
hidden tensor([[-1.9789e+01],
        [ 2.6903e-01],
        [ 2.9143e-03],
        [ 2.0708e-01],
        [-1.5959e+01],
        [-2.0320e-01],
        [-1.8764e+01],
        [-2.5711e+01]])
